In [1]:
import io
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from flask import Flask, request, abort, send_from_directory
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, ImageMessage, ImageSendMessage
from PIL import Image
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__)
line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info(f"Request body: {body}")

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_content = line_bot_api.get_message_content(event.message.id)
    image_data = io.BytesIO(message_content.content)
    image_path = 'input_image.jpg'
    with open(image_path, 'wb') as f:
        f.write(image_data.read())

    # 執行 Notebook
    nb_path = 'face(all_points,bfs,for_demo).ipynb'
    run_notebook(nb_path)

    # 假設 Notebook 處理完畢後保存圖片為 'output_image.jpg'
    output_image_path = 'output_image.jpg'
    line_bot_api.reply_message(
        event.reply_token,
        ImageSendMessage(
            original_content_url=f"https://{os.environ.get('kaomoji')}.herokuapp.com/images/{output_image_path}",
            preview_image_url=f"https://{os.environ.get('kaomoji')}.herokuapp.com/images/{output_image_path}"
        )
    )

def run_notebook(path):
    with open(path) as f:
        nb = nbformat.read(f, as_version=4)
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    ep.preprocess(nb)
    return nb

@app.route('/images/<filename>')
def uploaded_file(filename):
    return send_from_directory('images', filename)

if __name__ == "__main__":
    app.run()
